In [6]:
import numpy as np
from PIL import Image
from matplotlib import cm
import math
from Ray import *

def degrees_to_radians(doubledegrees):
    return degrees * pi / 180.0;

In [7]:
img = Image.new(mode = "RGB", size = (800, 600), color = 0)
array = np.array(img)

for j in reversed(range(array.shape[0])):
    if j % 50 == 0:
        print(j, "Scanlines remaining")
    for i in range(array.shape[1]):
        r = i / array.shape[0]
        g = j / array.shape[1]
        b = 0.25
        array[j, array.shape[1] - i  - 1] = [r * 255, g * 255, b * 255]
print("Done")

new_pic = Image.fromarray(array.astype('uint8'), 'RGB')
new_pic.show()

550 Scanlines remaining
500 Scanlines remaining
450 Scanlines remaining
400 Scanlines remaining
350 Scanlines remaining
300 Scanlines remaining
250 Scanlines remaining
200 Scanlines remaining
150 Scanlines remaining
100 Scanlines remaining
50 Scanlines remaining
0 Scanlines remaining
Done


In [8]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm
def color(array):
    c = np.multiply(255.0, array)

    return c
color(np.array([-1, 2, 0]))

array([-255.,  510.,    0.])

In [9]:
class Ray:
#     point3 origin
#     vec3 direction
    def __init__(self, origin, direction):
        self.origin = origin
        self.direction = direction
    def at(self,t):
        return self.origin + t * self.direction

In [31]:
# Vec3 center
# float radius
# Ray r
# for Ray A + tb, if exist t such that (A + tb - center) ^ 2 < r ^ 2, it hits the sphere.
# determine if root of t exists with the quadratic function t^2 b⋅b + 2tb⋅(A−C) + (A−C)⋅(A−C) − r ^ 2=0
def hit_sphere(center, radius, r):
    AC = np.subtract(r.origin, center)
    a = np.dot(r.direction, r.direction)
    half_b = np.dot(r.direction, AC)
    c = np.dot(AC, AC) - radius * radius
    delta = (half_b * half_b - a * c)
    if delta < 0 :
        return -1.0
    else :
        return (-half_b - math.sqrt(delta)) / a

In [42]:
# # point, normal, t, 
# class hit_record:
#     def __init_(self, p, normal, t):
#         self.p = p
#         self.normal = normal
#         self.t = t
#     def set_face_normal(self, r, outward_normal):
#         self.front_face = np.dot(r.direction, outward_normal) < 0
#         normal = front_face if outward_normal else -outward_normal;

        
class Hittable:
    def hit(ray, t_min, t_max, rec):
        pass

class Sphere:
#     Vec3 center
#     float radius
    def __init__(self, center, radius):
        self.center = center
        self.radius = radius
    
#     Hit function: return hit/non-hit for t in [t_min, t_max] and update hit_record if hit. 
    def hit(self, r, t_min, t_max, rec):
        AC = np.subtract(r.origin, self.center)
        a = np.dot(r.direction, r.direction)
        half_b = np.dot(r.direction, AC)
        c = np.dot(AC, AC) - radius * radius
        discriminant = (half_b * half_b - a * c)
        if (discriminant < 0) :
            return false
        
        sqrtd = math.sqrt(discriminant)
        root = (-half_b - sqrtd ) / a
        if t_min < root and root < t_max:
            root = (-half_b + sqrtd) / a
            if root < t_min or t_max < root:
                return false
        rec.t = root
        rec.p = r.at(rec.t)
        rec.normal = (rec.p - self.center) / self.radius
        return true


class Hittable_list(Hittable):
    def __init__(self):
        self.objects = []
    def add(obj):
        objects.append(obj)
    def clear():
        objects = []
    
    
    def hit(r, t_min, t_max, rec):
        temp_rec = None
        hit_anything = false
        temp_closest = t_max
        
        for obj in objects:
            if obj.hit(r, t_min, temp_closest, temp_rec):
                hit_anything = true
                rec = temp_rec
                
        return hit_anything
    

            
    

In [48]:
# return the color of a ray
# form: np.array(255, 255, 255)
def ray_color(r):
    
#     Detect Sphere hit
#     Sphere: C = (0, 0, -1), r = 0.5
    hitPoint = hit_sphere(np.array([0, 0, -1]), 0.5, r)
    if (hitPoint > 0):
        N = (np.subtract(r.at(hitPoint), np.array([0, 0, -1])))
        normalize(N)
        
        print(N)
        return np.multiply(0.5, color(np.array([N[0] + 1, N[1] + 1, N[2] + 1])))

    
#     No Sphere hit (Background)
    unit_direction = normalize(r.direction)
    t = 0.5 * (unit_direction[1] + 1)
    return (1.0 - t) * color(np.array([1, 1, 1])) + t * color(np.array([0.5, 0.7, 1.0]))

In [49]:
# main

# Image
image_width = 160
image_height = 120
aspect_ratio = image_width / image_height

# PIL preprocessing to use PIL coordinates
img = Image.new(mode = "RGB", size = (image_width, image_height), color = 0)
img_array = np.array(img)
img_array = np.swapaxes(img_array, 1, 0)

# Camera
viewport_height = 2
viewport_width = aspect_ratio * 2
focal_length = 1

#

origin = np.array([0, 0, 0])
horizontal = np.array([viewport_width, 0, 0])
vertical = np.array([0, viewport_height, 0])
lower_left_corner = origin - horizontal / 2 - vertical / 2 - np.array([0, 0, focal_length])

# Render
print("image width: ", image_width," image height: ", image_height)
for j in reversed(range(image_height)):
    if j % 50 == 0:
        print(j, "Scanlines remaining")
    for i in range(image_width):
        u = i / (image_width - 1)
        v = j / (image_height - 1)
        r = Ray(origin, lower_left_corner + u * horizontal + v * vertical - origin)
        img_array[i][j] = ray_color(r)

print("Done")

image width:  160  image height:  120
100 Scanlines remaining
[-0.09252338  0.41413932  0.26443913]
[-0.07729473  0.39920204  0.29096951]
[-0.06425898  0.39221786  0.30337424]
[-0.05196197  0.38764069  0.31150385]
[-0.04008393  0.3844665   0.31714159]
[-0.02846966  0.38229495  0.32099851]
[-0.01702077  0.38092923  0.3234242 ]
[-0.00566375  0.38026825  0.32459818]
[0.00566375 0.38026825 0.32459818]
[0.01702077 0.38092923 0.3234242 ]
[0.02846966 0.38229495 0.32099851]
[0.04008393 0.3844665  0.31714159]
[0.05196197 0.38764069 0.31150385]
[0.06425898 0.39221786 0.30337424]
[0.07729473 0.39920204 0.29096951]
[0.09252338 0.41413932 0.26443913]
[-0.12524372  0.38847354  0.2887946 ]
[-0.11015347  0.3776325   0.30864204]
[-0.09677236  0.37078928  0.32117039]
[-0.08424203  0.36581569  0.33027588]
[-0.07225201  0.36201899  0.33722678]
[-0.06063851  0.35907123  0.34262344]
[-0.04929801  0.35678916  0.34680139]
[-0.03815717  0.35506092  0.34996538]
[-0.02715951  0.35381536  0.35224573]
[-0.01625848

[0.08152656 0.2450935  0.42811517]
[0.09154772 0.24624959 0.42541762]
[0.1017291  0.2475754  0.42232406]
[0.11209825 0.24908788 0.41879495]
[0.1226875  0.2508083  0.41478063]
[0.13353557 0.25276375 0.41021792]
[0.14468992 0.25498926 0.40502507]
[0.15621036 0.25753113 0.39909402]
[0.16817472 0.26045241 0.39227772]
[0.18068857 0.26384218 0.38436824]
[0.19390292 0.26783309 0.37505612]
[0.2080504  0.27263747 0.36384591]
[0.22353086 0.27863472 0.34985233]
[0.24117218 0.28664235 0.33116784]
[0.26356943 0.29933957 0.301541  ]
[-0.27380696  0.28605802  0.30528767]
[-0.25120603  0.27411011  0.33430402]
[-0.23331454  0.26642861  0.35295909]
[-0.21762683  0.26063701  0.3670244 ]
[-0.20331078  0.25597839  0.3783382 ]
[-0.18995913  0.252096    0.38776686]
[-0.17733354  0.24878852  0.3957993 ]
[-0.16527878  0.24592952  0.40274259]
[-0.15368622  0.24343372  0.40880383]
[-0.14247573  0.2412406   0.41412998]
[-0.13158581  0.23930556  0.41882935]
[-0.1209677   0.23759479  0.42298408]
[-0.11058171  0.236

[-0.08679714  0.18769258  0.45523373]
[-0.07737921  0.18701251  0.45720759]
[-0.06805926  0.18641946  0.45892889]
[-0.05882319  0.18590905  0.46041031]
[-0.04965798  0.18547768  0.46166234]
[-0.04055143  0.18512241  0.4626935 ]
[-0.03149204  0.18484088  0.46351063]
[-0.0224688   0.18463127  0.464119  ]
[-0.01347113  0.18449225  0.46452248]
[-0.00448869  0.18442296  0.4647236 ]
[0.00448869 0.18442296 0.4647236 ]
[0.01347113 0.18449225 0.46452248]
[0.0224688  0.18463127 0.464119  ]
[0.03149204 0.18484088 0.46351063]
[0.04055143 0.18512241 0.4626935 ]
[0.04965798 0.18547768 0.46166234]
[0.05882319 0.18590905 0.46041031]
[0.06805926 0.18641946 0.45892889]
[0.07737921 0.18701251 0.45720759]
[0.08679714 0.18769258 0.45523373]
[0.09632844 0.18846492 0.45299206]
[0.10599013 0.18933585 0.45046424]
[0.1158012  0.19031295 0.44762826]
[0.12578318 0.19140536 0.44445763]
[0.1359607  0.19262413 0.44092021]
[0.14636243 0.19398278 0.43697679]
[0.15702217 0.195498   0.43257898]
[0.16798057 0.19719064 0.

[0.13089742 0.14021879 0.46174079]
[0.14074585 0.14104154 0.45858248]
[0.15079106 0.1419498  0.45509594]
[0.16105936 0.14295227 0.45124775]
[0.17158153 0.14405951 0.44699735]
[0.18239411 0.14528452 0.44229492]
[0.19354119 0.14664345 0.43707838]
[0.20507699 0.1481568  0.43126905]
[0.21706979 0.14985112 0.42476505]
[0.22960793 0.15176169 0.41743094]
[0.24280969 0.15393701 0.40908049]
[0.25684059 0.15644677 0.39944626]
[0.2719463  0.1593971  0.38812083]
[0.28852331 0.16296388 0.37442899]
[0.30730022 0.1674793  0.35709558]
[0.32997046 0.17373854 0.33306819]
[0.36418628 0.18546709 0.28804568]
[-0.35005169  0.16676764  0.31567763]
[-0.32326443  0.15922649  0.34662232]
[-0.3026068   0.15428129  0.36691471]
[-0.28486856  0.15051898  0.38235316]
[-0.26894818  0.14746949  0.39486659]
[-0.25430376  0.14490789  0.40537797]
[-0.24061889  0.14270627  0.4144122 ]
[-0.22768867  0.1407839   0.42230056]
[-0.21537081  0.1390861   0.42926737]
[-0.20356132  0.13757396  0.43547238]
[-0.19218123  0.13621864 

[0.01278685 0.08969601 0.49172261]
[0.02132425 0.08975002 0.49141653]
[0.02988101 0.08983136 0.49095565]
[0.03846507 0.08994039 0.49033781]
[0.04708461 0.09007764 0.48956007]
[0.05574807 0.09024376 0.48861868]
[0.06446427 0.09043959 0.48750901]
[0.07324249 0.0906661  0.48622546]
[0.08209253 0.09092447 0.48476134]
[0.09102487 0.0912161  0.48310878]
[0.10005076 0.09154261 0.48125855]
[0.10918242 0.09190591 0.47919986]
[0.11843319 0.09230822 0.47692009]
[0.12781777 0.09275214 0.47440453]
[0.1373525  0.09324072 0.47163594]
[0.14705573 0.09377752 0.46859405]
[0.15694823 0.09436677 0.46525494]
[0.16705379 0.0950135  0.46159015]
[0.17739995 0.09572373 0.45756554]
[0.18801902 0.09650474 0.45313981]
[0.19894945 0.09736548 0.44826229]
[0.21023773 0.09831706 0.44287002]
[0.22194117 0.09937354 0.4368833 ]
[0.23413202 0.1005531  0.4301991 ]
[0.24690385 0.10187988 0.42268071]
[0.260382   0.10338697 0.41414049]
[0.27474187 0.10512182 0.40430966]
[0.29024346 0.10715645 0.39278013]
[0.30730516 0.109609

...

[ 0.10850034 -0.07393523  0.48245337]
[ 0.11767964 -0.07425025  0.48024827]
[ 0.12698785 -0.07459754  0.47781722]
[ 0.13644048 -0.07497939  0.47514428]
[ 0.14605487 -0.07539846  0.47221081]
[ 0.15585056 -0.07585788  0.46899487]
[ 0.16584983 -0.07636136  0.46547049]
[ 0.1760783  -0.07691332  0.46160673]
[ 0.18656587 -0.07751909  0.45736634]
[ 0.19734782 -0.07818514  0.45270401]
[ 0.20846644 -0.07891944  0.44756392]
[ 0.21997334 -0.07973198  0.44187616]
[ 0.2319327  -0.0806355   0.43555153]
[ 0.24442642 -0.08164664  0.42847351]
[ 0.25756204 -0.0827878   0.4204854 ]
[ 0.27148621 -0.08409021  0.41136854]
[ 0.28640885 -0.08559964  0.40080253]
[ 0.3026512  -0.08738779  0.38828549]
[ 0.32075498 -0.08957853  0.3729503 ]
[ 0.34178979 -0.09242275  0.35304076]
[ 0.36864911 -0.09661848  0.32367067]
50 Scanlines remaining
[-0.37514269 -0.10988747  0.31175745]
[-0.34550285 -0.10441818  0.34601246]
[-0.32345326 -0.10095939  0.36767538]
[-0.30478851 -0.09835843  0.3839656 ]
[-0.28818016 -0.09626186  0

[ 0.16005634 -0.13289672  0.45466518]
[ 0.17047396 -0.13389543  0.45056702]
[ 0.18116868 -0.13499819  0.44604193]
[ 0.19218123 -0.13621864  0.44103385]
[ 0.20356132 -0.13757396  0.43547238]
[ 0.21537081 -0.1390861   0.42926737]
[ 0.22768867 -0.1407839   0.42230056]
[ 0.24061889 -0.14270627  0.4144122 ]
[ 0.25430376 -0.14490789  0.40537797]
[ 0.26894818 -0.14746949  0.39486659]
[ 0.28486856 -0.15051898  0.38235316]
[ 0.3026068  -0.15428129  0.36691471]
[ 0.32326443 -0.15922649  0.34662232]
[ 0.35005169 -0.16676764  0.31567763]
[-0.36418628 -0.18546709  0.28804568]
[-0.32997046 -0.17373854  0.33306819]
[-0.30730022 -0.1674793   0.35709558]
[-0.28852331 -0.16296388  0.37442899]
[-0.2719463  -0.1593971   0.38812083]
[-0.25684059 -0.15644677  0.39944626]
[-0.24280969 -0.15393701  0.40908049]
[-0.22960793 -0.15176169  0.41743094]
[-0.21706979 -0.14985112  0.42476505]
[-0.20507699 -0.1481568   0.43126905]
[-0.19354119 -0.14664345  0.43707838]
[-0.18239411 -0.14528452  0.44229492]
[-0.17158153

[ 0.05837306 -0.17548708  0.46453942]
[ 0.06753206 -0.17595223  0.46312011]
[ 0.07677125 -0.17649228  0.46147226]
[ 0.08610396 -0.177111    0.45958438]
[ 0.09554469 -0.17781291  0.45744265]
[ 0.10510941 -0.17860342  0.45503058]
[ 0.11481584 -0.179489    0.45232845]
[ 0.12468393 -0.18047738  0.44931262]
[ 0.13473639 -0.18157788  0.44595468]
[ 0.14499937 -0.18280179  0.44222018]
[ 0.15550347 -0.18416292  0.43806699]
[ 0.16628508 -0.18567835  0.43344299]
[ 0.17738817 -0.18736952  0.42828273]
[ 0.18886709 -0.18926387  0.42250256]
[ 0.20079065 -0.19139724  0.41599304]
[ 0.21324865 -0.19381789  0.40860695]
[ 0.22636273 -0.19659317  0.40013878]
[ 0.24030599 -0.19982176  0.39028745]
[ 0.25534179 -0.20365818  0.37858145]
[ 0.27191181 -0.20836939  0.36420621]
[ 0.29088409 -0.21449646  0.34551081]
[ 0.31460281 -0.22355065  0.3178839 ]
[-0.33330709 -0.24898727  0.27732963]
[-0.2994277  -0.23211937  0.32628769]
[-0.27789087 -0.2238718   0.35022576]
[-0.25999593 -0.21800465  0.3672548 ]
[-0.24411978

[-0.25491059 -0.3148522   0.29306771]
[-0.23186401 -0.30035632  0.32561505]
[-0.21423041 -0.29174526  0.34494932]
[-0.19882927 -0.28540784  0.35917863]
[-0.1847673  -0.28037828  0.37047141]
[-0.17163001 -0.27622729  0.37979155]
[-0.15918109 -0.27272006  0.38766629]
[-0.14726888 -0.269712    0.39442023]
[-0.1357881  -0.26710699  0.40026922]
[-0.12466147 -0.26483754  0.40536477]
[-0.11382976 -0.26285441  0.40981745]
[-0.10324596 -0.26112056  0.41371043]
[-0.09287174 -0.25960753  0.41710763]
[-0.08267503 -0.25829307  0.42005896]
[-0.07262844 -0.25715962  0.42260388]
[-0.0627081  -0.25619321  0.42477375]
[-0.05289284 -0.25538273  0.42659349]
[-0.04316356 -0.2547194   0.42808286]
[-0.03350272 -0.25419635  0.42925724]
[-0.02389399 -0.25380839  0.43012833]
[-0.0143219  -0.25355177  0.43070452]
[-0.00477156 -0.25342405  0.43099128]
[ 0.00477156 -0.25342405  0.43099128]
[ 0.0143219  -0.25355177  0.43070452]
[ 0.02389399 -0.25380839  0.43012833]
[ 0.03350272 -0.25419635  0.42925724]
[ 0.04316356

[-0.09252338 -0.41413932  0.26443913]
[-0.07729473 -0.39920204  0.29096951]
[-0.06425898 -0.39221786  0.30337424]
[-0.05196197 -0.38764069  0.31150385]
[-0.04008393 -0.3844665   0.31714159]
[-0.02846966 -0.38229495  0.32099851]
[-0.01702077 -0.38092923  0.3234242 ]
[-0.00566375 -0.38026825  0.32459818]
[ 0.00566375 -0.38026825  0.32459818]
[ 0.01702077 -0.38092923  0.3234242 ]
[ 0.02846966 -0.38229495  0.32099851]
[ 0.04008393 -0.3844665   0.31714159]
[ 0.05196197 -0.38764069  0.31150385]
[ 0.06425898 -0.39221786  0.30337424]
[ 0.07729473 -0.39920204  0.29096951]
[ 0.09252338 -0.41413932  0.26443913]
0 Scanlines remaining
Done


In [50]:
final_img_array = np.swapaxes(img_array, 1, 0)
final_img_array = np.flip(final_img_array, 0)
new_pic = Image.fromarray(final_img_array.astype('uint8'), 'RGB')
new_pic.show()